# Almost Visual Inertial Odometry

# Imports

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# Constants

In [ ]:
SEED = 20197
BATCH_SIZE = 16
TEST_BATCH_SIZE = 32
GAMMA = 0.1   # torch default
LR=0.001 
EPOCHS = 50

# Device

In [ ]:
available_cuda = torch.cuda.is_available()
print(f"Available cuda: {available_cuda}")

device = torch.device("cuda" if available_cuda else "cpu")

# Dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):
      def __init__(self, images, labels):
            self.labels = labels
            self.images = images

      def __len__(self):
            return len(self.images)

      def __getitem__(self, index):
            image_name = self.images[index]

            X = torch.load('data/' + image_name + '.png')
            y = self.labels[image_name]

            return X, y

# Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        pass

    def forward(self, x):
        pass

# Training

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(dataset):
        # data preparation
        data, target = data.to(device), target.to(device)

        #forward
        optimizer.zero_grad()
        output = model(data)

        # loss
        loss = torch.sqrt(F.mse_loss(output, data))

        # backward
        loss.backward()
        optimizer.step()

        #stats
        history_loss += loss.item()

    return history_loss/(len(train_loader.dataset)/BATCH_SIZE) # average epoch loss


In [ ]:
def test(model, device, train_loader, optimizer, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(dataset):
        # data preparation
        data, target = data.to(device), target.to(device)

        # forward
        optimizer.zero_grad()
        output = model(data)

        # loss
        loss = torch.sqrt(F.mse_loss(output, data))

        #stats
        history_loss += loss.item()

    return history_loss/(len(train_loader.dataset)/BATCH_SIZE) # average epoch loss


In [ ]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

h_loss_train = []
h_loss_test = []

for epoch in range(1, EPOCHS + 1):
    print(f"Epoch {epoch}/{EPOCHS}")

    loss_train = train(model, device, train_loader, optimizer, epoch)
    loss_test, data, output, embedded_code = test(model, device, test_loader)

    #populating the history of the loss
    h_loss_train.append(loss_train)
    h_loss_test.append(loss_test)

# Plotting